In [9]:
tb_data = tb.get_parsed_metadata()

In [10]:
tb_data.keys()

dict_keys(['parameters', 'element_spec', 'context_spec', 'raw'])

In [11]:
import pprint
from backend.manager.tensorboard_manager import TensorBoardManager

tb = TensorBoardManager()
tb.set_work_dir(
    "/home/halcyon/root/gemma_fine_tuning_ui/checkpoints/Gemma3_1B-20250630_223543"
)

tb
total_steps = 100
kpi = {
    # Metadata
    **tb.get_parsed_metadata()["parameters"],
    # Progress KPIs
    "current_step": tb.get_current_step(),
    "total_steps": total_steps,
    "current_loss": tb.get_current_loss(),
    "training_time": tb.get_training_time(),
    # Performance KPIs
    "training_speed": tb.get_training_speed(),
    "data_throughput": tb.get_data_throughput(),
    "avg_step_time": tb.get_avg_step_time(),
    "avg_eval_time": tb.get_avg_eval_time(),
    "eta_str": tb.get_eta_str(total_steps),
}
print(kpi.get("total_params") or kpi.get("total_bytes"))

999885952


In [14]:
kpi.get("total_bytes")

1999771904

In [13]:
print(kpi.keys())

dict_keys(['total_params', 'total_bytes', 'layers', 'parameter_count', 'current_step', 'total_steps', 'current_loss', 'training_time', 'training_speed', 'data_throughput', 'avg_step_time', 'avg_eval_time', 'eta_str'])


In [16]:
len(kpi.get("layers"))

286

In [18]:
kpi.get("parameter_count")

697894912

In [17]:
kpi.get("total_bytes")/1024**3

1.8624327182769775

In [12]:
has_metadata = kpi.get("total_params") or kpi.get("total_memory_mb")
is_training = kpi.get("current_step", 0) > 0

if not has_metadata or not is_training:
    print("Waiting for training data...")


if is_training:
    print("#### Training Progress")
    total_steps = kpi.get("total_steps", 0)
    current_step = kpi.get("current_step", 0)
    step_str = (
        f"{current_step}/{total_steps}"
        if total_steps > 0
        else str(current_step)
    )

    print("Global Step", step_str)
    print("Current Loss", f"{kpi.get('current_loss', 0.0):.4f}")
    print(
        "Training Speed",
        f"{kpi.get('training_speed', 0.0):.2f} steps/sec",
    )
    print("Training Time", kpi.get("training_time", "00:00:00"))

    print("#### Performance Metrics")
    print(
        "Data Throughput",
        f"{kpi.get('data_throughput', 0.0):.0f} samples/sec",
    )
    print("ETA", kpi.get("eta_str", "N/A"))
    print("Avg Step Time", f"{kpi.get('avg_step_time', 0.0):.3f}s")
    print("Avg Eval Time", f"{kpi.get('avg_eval_time', 0.0):.3f}s")

elif has_metadata:
    # If we have metadata but training hasn't started, show a clear message.
    print("Waiting for training to start to show progress KPIs...")

#### Training Progress
Global Step 11.0/100
Current Loss 3.9699
Training Speed 0.08 steps/sec
Training Time 0.10511519014835358
#### Performance Metrics
Data Throughput 1 samples/sec
ETA 00:17:35
Avg Step Time 11.865s
Avg Eval Time 0.000s
